<a href="https://colab.research.google.com/github/VisheshJain112/news_effect_on_stock_prices/blob/master/Stock_Sentiment_prediction_from_news_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

In [4]:
with open('/stocktwit.json', 'r') as f:
    twits = json.load(f)
import re
print(twits)
"""cnt=0
for twit in twits:
  try:
    print(twit['message'])
  except KeyError:
    cnt+=1
    print(twit)
    pass
"""
tweet=[]
tweet_sentiment=[]
for twit in twits:
  try:
    tweet.append(twit['message']['body'])
    tweet_sentiment.append(twit['sentiment score'])
  except KeyError:
    pass




[{'source': 'stocktwits', 'cashtag': '$NFLX', 'sentiment score': '-0.494', 'id': '5329774', 'spans': ['out $NFLX -.35'], 'response': {'status': 200}, 'message': {'id': 5329774, 'body': 'out $NFLX -.35', 'created_at': '2011-10-05T15:49:12Z', 'user': {'id': 284, 'username': 'skayfe', 'name': 'Jared Schafer', 'avatar_url': 'https://avatars.stocktwits.com/production/284/thumb-1326124844.png', 'avatar_url_ssl': 'https://avatars.stocktwits.com/production/284/thumb-1326124844.png', 'join_date': '2009-09-01', 'official': False, 'identity': 'User', 'classification': [], 'followers': 84, 'following': 26, 'ideas': 3768, 'watchlist_stocks_count': 3, 'like_count': 0, 'plus_tier': '', 'premium_room': ''}, 'source': {'id': 1, 'title': 'Stocktwits', 'url': 'https://stocktwits.com'}, 'symbols': [{'id': 2839, 'symbol': 'NFLX', 'title': 'Netflix, Inc.', 'aliases': [], 'is_following': False, 'watchlist_count': 190457}], 'mentioned_users': [], 'entities': {'sentiment': None}, 'liked_by_self': False, 'resha

In [40]:
with open('/stocktwit.json', 'r') as f:
    twits = json.load(f)
import re
#print(twits)
"""cnt=0
for twit in twits:
  try:
    print(twit['message'])
  except KeyError:
    cnt+=1
    print(twit)
    pass
"""
tweet_time=[]
for twit in twits:
  try:
    tweet_time.append(twit['message']['created_at'])
    #tweet_sentimen.append(twit['sentiment score'])
  except KeyError:
    pass



919


In [24]:
#with open('/tweet_stock.json', 'r') as f:
 #   sh = f.read()



In [5]:
with open('/twitter.json', 'r') as f:
    twits = json.load(f)
import re

"""cnt=0
for twit in twits:
  try:
    print(twit['message'])
  except KeyError:
    cnt+=1
    print(twit)
    pass
"""
for twit in twits:
  print(twit)
  try:
    tweet.append(twit['text'])
    tweet_sentiment.append(twit['sentiment score'])
  except KeyError:
    pass

{'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'cashtag': '$FB', 'sentiment score': '0.366', 'id': 719659409228451840, 'spans': ['watching for bounce tomorrow'], 'created_at': 'Mon Apr 11 22:52:28 +0000 2016', 'id_str': '719659409228451840', 'text': '$FB watching for bounce tomorrow', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [{'text': 'FB', 'indices': [0, 3]}], 'user_mentions': [], 'urls': []}, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1162268034, 'id_str': '1162268034', 'name': 'Heather', 'screen_name': 'calitrade', 'location': 'Huntington Beach, CA', 'description': 'Trading since I was 16. Swing trading for 8 years. Day trading for 4 years. I have an over 90% win rate on swings. I always put my money to work.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_cou

In [43]:
with open('/twitter.json', 'r') as f:
    twits = json.load(f)
import re

"""cnt=0
for twit in twits:
  try:
    print(twit['message'])
  except KeyError:
    cnt+=1
    print(twit)
    pass
"""
for twit in twits:
  
  try:
    tweet_time.append(twit['message']['created_at'])
    #tweet_sentiment.append(twit['sentiment score'])
  except KeyError:
    pass

1454


In [7]:
for i,val in enumerate(tweet_sentiment):
  val=float(val)
  if (float(val)< float(-0.5)):
    tweet_sentiment[i]=0
  elif (float(val)>-0.5 and val<0):
    tweet_sentiment[i]=1
  elif (val>0 and val<0.5):
    tweet_sentiment[i]=3
  elif (val>0.5 and val<=1):
    tweet_sentiment[i]=4
  else:
    tweet_sentiment[i]=2
  




In [8]:
nltk.download('wordnet')
nltk.download('punkt')

def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'[$][A-Za-z][\S]*', ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'[@][A-Za-z][\S]*', ' ', text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^A-Za-z]', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    #tokens = [word for word in text.split(' ') if len(word) > 0]
    tokens = nltk.word_tokenize(text)
    
    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(word) for word in tokens if len(word) > 1]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
tokenized = [preprocess(message) for message in tweet]

In [10]:
see=[' '.join(token) for token in tokenized]

In [11]:
vocab_text=' '.join(see)
vocab=vocab_text.split()

In [12]:
from collections import Counter
bow=Counter(vocab)

In [13]:

import operator
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {word:bow[word]/len(bow) for id, word in enumerate(bow)}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.000001

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 15

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [x[0] for x in sorted(freqs.items(), key=operator.itemgetter(1))[-high_cutoff:]]

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)

['with', 'be', 'at', 'it', 'this', 'up', 'stock', 'in', 'is', 'on', 'for', 'of', 'and', 'to', 'the']


In [14]:
vocab={word : idx for idx,word in enumerate(filtered_words,start=1)}

In [15]:
id2vocab = {idx:word for word, idx in vocab.items()}

In [16]:
filtered = [[token for token in message if token in vocab] for message in tokenized]

In [17]:
print(filtered)

[['out'], ['been', 'doing', 'some', 'work', 'evening', 'very', 'intrigued', 'technology', 'growth', 'potential'], ['my', 'short', 'worked', 'thank', 'you', 'hint', 'put'], ['meanwhile', 'absolute', 'garbage', 'still', 'store', 'totally', 'empty', 'inventory', 'galore', 'mispriced'], ['who', 'would', 'guessed', 'would', 'kill', 'would', 'suck', 'not', 'me', 'lol'], ['heavy', 'volume', 'alert', 'split', 'defensive', 'cyclical', 'sector', 'no', 'edge', 'offered'], ['run', 'into', 'sma', 'acquisition', 'news'], ['make', 'sure', 'you', 'wait', 'don', 'settle', 'that', 'would', 'throwing', 'money', 'away', 'can', 'go', 'down'], ['china', 'wa', 'always', 'aapl', 'ace', 'sleave', 'now', 'seems', 'like', 'helping', 'downtrend'], ['rt', 'since', 'mastered', 'their', 'supply', 'chain', 'cornering', 'nand', 'apple', 'mastered', 'everyone', 'supply', 'chain'], ['big', 'dumping', 'would', 'not', 'touch', 'while'], ['trade', 'continuing', 'very', 'nicely', 'from', 'yesterday', 'almost', 'from', 'yest

In [19]:
tweet_int=[[vocab[word] for word in filt] for i,filt in enumerate(filtered)]

In [22]:
print(tweet_int)

[[1], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24, 25, 26, 27, 28], [29, 30, 31, 30, 32, 30, 33, 34, 35, 36], [37, 38, 39, 40, 41, 42, 43, 44, 45, 46], [47, 48, 49, 50, 51], [52, 53, 16, 54, 55, 56, 57, 30, 58, 59, 60, 61, 62, 63], [64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74], [75, 76, 77, 78, 79, 80, 81, 82, 83, 77, 84, 79, 80], [85, 86, 30, 34, 87, 88], [89, 90, 7, 91, 92, 93, 94, 92, 93, 95, 96, 97, 98, 99, 7, 100, 101], [102, 103, 104, 105, 106, 107, 108, 109], [110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 115, 122, 113], [123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 127, 133], [134, 135, 85, 136], [137, 138, 139, 140], [141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151], [152, 128, 153, 154, 155, 100, 156, 157, 103, 13, 158], [159, 160, 161, 162, 163, 164, 165, 166, 135, 167, 168, 169, 170, 171, 172], [18, 173, 1, 174, 175, 147, 176, 177, 178, 63, 179], [180, 181, 182, 183, 183, 184], [113, 185, 186, 187, 61, 188,

In [70]:
import numpy as np
def padded_features(vocab_int,seq_length):
  features=np.zeros((len(vocab_int),seq_length),dtype=int)
  print(features.shape)
  for i,text in enumerate(vocab_int):
    sati=np.array(text)[:seq_length]
    print(sati)
    
    features[i,-len(text):]=sati
  return features


In [72]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        print(features[i, -len(row):]) 
        np.array(row)[:seq_length]
    
    return features

In [127]:
def dataloader(messages, labels, sequence_length=32, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

In [128]:
split_perc = 0.90

# TODO Implement 
split_idx = int(len(tweet_int)*split_perc)

train_features = tweet_int[:split_idx]
valid_features = tweet_int[split_idx:]
train_labels = tweet_sentiment[:split_idx]
valid_labels = tweet_sentiment[split_idx:]

In [129]:
trainloader=dataloader(train_features,train_labels)

In [130]:
validloader=dataloader(valid_features,valid_labels)

In [131]:
device=torch.device("cuda")

AttributeError: ignored

In [71]:

seq_length = 20

features = padded_features(tweet_int, seq_length)

(1454, 20)
[1]
[ 2  3  4  5  6  7  8  9 10 11]
[12 13 14 15 16 17 18]
[19 20 21 22 23 24 25 26 27 28]
[29 30 31 30 32 30 33 34 35 36]
[37 38 39 40 41 42 43 44 45 46]
[47 48 49 50 51]
[52 53 16 54 55 56 57 30 58 59 60 61 62 63]
[64 65 66 67 68 69 70 71 72 73 74]
[75 76 77 78 79 80 81 82 83 77 84 79 80]
[85 86 30 34 87 88]
[ 89  90   7  91  92  93  94  92  93  95  96  97  98  99   7 100 101]
[102 103 104 105 106 107 108 109]
[110 111 112 113 114 115 116 117 118 119 120 121 115 122 113]
[123 124 125 126 127 128 129 130 131 132 127 133]
[134 135  85 136]
[137 138 139 140]
[141 142 143 144 145 146 147 148 149 150 151]
[152 128 153 154 155 100 156 157 103  13 158]
[159 160 161 162 163 164 165 166 135 167 168 169 170 171 172]
[ 18 173   1 174 175 147 176 177 178  63 179]
[180 181 182 183 183 184]
[113 185 186 187  61 188 189 190 113 191]
[123 124 125 126 127 128 129 130 131 132 127 133]
[192 193  16  61  13 101 194 195 196 197 198 199  72 200 201 202 203]
[ 61 204  13 205  62  48 206 207 208 

ValueError: ignored

In [50]:
features=padded_features(tweet_int,50)

ValueError: ignored

In [144]:

# First checking if GPU is available
train_on_gpu = torch.cuda.is_available()

if(train_on_gpu):
    print('[ info ] Training_on_gpu.')
else:
    print('[ info ] No GPU available, training in CPU.')
from torch import nn
class TextClassifer(nn.Module):
  
  def __init__(self,input_size,embed_dim,n_layer,hidden_dim,output_dim,drop_prob):
    super().__init__()
    self.input_size=input_size
    self.embed_dim=embed_dim
    self.hidden_dim=hidden_dim
    self.output_dim=output_dim
    self.drop_prob=drop_prob
    self.n_layer=n_layer
    self.embed=nn.Embedding(input_size,embed_dim)
    self.lstm=nn.LSTM(embed_dim,hidden_dim,n_layer,dropout=drop_prob,batch_first=True)
    self.dropout=nn.Dropout(0.3)
    self.fc=nn.Linear(hidden_dim,output_dim)
    self.softmax=nn.LogSoftmax()
  def forward(self,x,hidden):
    batch_size=x.shape[0]
    x=x.long()
    embeds=self.embed(x)
    output,hidden=self.lstm(embeds,hidden)
    
    output = output[-1,:,:]
    output=self.dropout(output)
    output=self.softmax(self.fc(output))
    return output,hidden
  def init_hidden(self,batch_size):
    weight=next(self.parameters()).data
    hidden = (weight.new(self.n_layer, batch_size, self.hidden_dim).zero_(), 
                    weight.new(self.n_layer, batch_size, self.hidden_dim).zero_())
          
    return hidden

[ info ] Training_on_gpu.


In [145]:
n_layer=2
output=5
embed_dim=1024
input_size=len(vocab)+1
hidden_dim=512
drop_prob=0.5


model=TextClassifer(input_size,embed_dim,n_layer,hidden_dim,output,drop_prob)

In [123]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=20)))

In [26]:
train_on_gpu = torch.cuda.is_available()

if(train_on_gpu):
    print('[ info ] Training_on_gpu.')
else:
    print('[ info ] No GPU available, training in CPU.')

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, 
                            dropout = dropout, batch_first = False)

        
        # Setup additional layers
        # drop out here
        self.drp = nn.Dropout(dropout)
        # linear and and softmax layers
        self.fc = nn.Linear(self.lstm_size, self.output_size)
        self.soft = nn.LogSoftmax()


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        #if (train_on_gpu):
        #    hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
        #              weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda()) 
        #else:
        
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(), 
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
            
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement
        batch_size = nn_input.size(1)
        
        #embeddings and lstm_out
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden_state)
        
        # stack up lstm outputs
        #lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
        lstm_out = lstm_out[-1,:,:]
        
        # dropout later
        out = self.drp(lstm_out)
        
        # fully connected layer
        out = self.fc(out)
        
        # softmax function
        soft_out = self.soft(out)
        
        # reshape the batch size first
        #soft_out = soft_out.view(batch_size, -1)
        #soft_out = soft_out[:, -1] # get last batch of labels
        
        return soft_out, hidden

[ info ] Training_on_gpu.


In [27]:

model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)
logps, _ = model.forward(input, hidden)
print('[ info ] logps', logps)

[ info ] logps tensor([[-1.8256, -1.9427, -1.5328, -1.5200, -1.3436],
        [-1.8605, -1.8928, -1.5275, -1.5464, -1.3331],
        [-1.8132, -1.9218, -1.4297, -1.6400, -1.3581],
        [-1.8510, -1.9359, -1.5259, -1.5410, -1.3204]],
       grad_fn=<LogSoftmaxBackward>)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:113: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [28]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

In [29]:
split_perc = 0.9

# TODO Implement 
token_ids=tweet_int
sentiments=tweet_sentiment
split_idx = int(len(token_ids)*split_perc)

train_features = token_ids[:split_idx]
valid_features = token_ids[split_idx:]
train_labels = sentiments[:split_idx]
valid_labels = sentiments[split_idx:]

In [30]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:113: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [31]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(2797, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (drp): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (soft): LogSoftmax()
)

In [33]:
import numpy as np

epochs = 3
batch_size = 100
learning_rate = 0.001

print_every = 10
criterion = nn.NLLLoss() # 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size):
        steps += 1
        hidden = model.init_hidden(labels.shape[0])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        # zero accumulated gradients
        model.zero_grad()
        
        # Get the output from the model
        output, hidden = model(text_batch, hidden)
        
        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels)
        loss.backward()
        
        # clip_grad_norm helps prevent the exploding gradient problem in RNNs
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        
        if steps % print_every == 0:
            # Get validation loss
            model.eval()
            val_losses = [] # Keep track of the losses for iteration...
            for inputs, labels in dataloader(valid_features, valid_labels, batch_size = batch_size):
                # move the inputs and labels to the relevant device
                inputs, labels = inputs.to(device), labels.to(device)
                
                val_h = model.init_hidden(labels.shape[0])
                
                for each in val_h:
                    each.to(device)
                
                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels)
                val_losses.append(val_loss.item())
                
            # TODO Implement: Print metrics
            model.train()
            print('[ info ] Epoch: {}/{}...'.format(epoch + 1, epochs),
                 '[ info ] Step: {}...'.format(steps),
                 '[ info ] Loss: {:.6f}...'.format(loss.item()),
                 '[ info ] Validation Loss: {:.6f}'.format(np.mean(val_losses)))

Starting epoch 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:113: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[ info ] Epoch: 1/3... [ info ] Step: 10... [ info ] Loss: 1.373679... [ info ] Validation Loss: 1.115488
Starting epoch 2
[ info ] Epoch: 2/3... [ info ] Step: 10... [ info ] Loss: 0.750591... [ info ] Validation Loss: 1.075907
Starting epoch 3
[ info ] Epoch: 3/3... [ info ] Step: 10... [ info ] Loss: 0.393516... [ info ] Validation Loss: 0.839984


In [34]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words, just like above
    tokens = [token for token in tokens if token in vocab]
    
    # Convert words to ids
    tokens = [vocab[token] for token in tokens]
        
    # Adding a batch dimension
    text_input = torch.FloatTensor(tokens).view(-1,1)
    text_input = torch.from_numpy(np.asarray(text_input))
    
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model.forward(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [35]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")


tensor([0.0091, 0.0141, 0.0085, 0.7997, 0.1686], grad_fn=<AddBackward0>)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:113: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [47]:
import pandas as pd
df=pd.read_csv(r'/stockerbot-export.csv', error_bad_lines=False)

b'Skipping line 731: expected 8 fields, saw 13\nSkipping line 2836: expected 8 fields, saw 15\nSkipping line 3058: expected 8 fields, saw 12\nSkipping line 3113: expected 8 fields, saw 12\nSkipping line 3194: expected 8 fields, saw 17\nSkipping line 3205: expected 8 fields, saw 17\nSkipping line 3255: expected 8 fields, saw 17\nSkipping line 3520: expected 8 fields, saw 17\nSkipping line 4078: expected 8 fields, saw 17\nSkipping line 4087: expected 8 fields, saw 17\nSkipping line 4088: expected 8 fields, saw 17\nSkipping line 4499: expected 8 fields, saw 12\n'


In [53]:
df['text'].values

array(['VIDEO: “I was in my office. I was minding my own business...” –David Solomon tells $GS interns how he learned he wa… https://t.co/QClAITywXV',
       "The price of lumber $LB_F is down 22% since hitting its YTD highs. The Macy's $M turnaround is still happening.… https://t.co/XnKsV4De39",
       'Who says the American Dream is dead? https://t.co/CRgx19x7sA',
       ...,
       "RT @invest_in_hd: 'Nuff said!  $TEL #telcoin #Telfam #crypto #Blockchain #ethereum #bitcoin $BTC $ETH https://t.co/dkRvaYzgcd",
       '【仮想通貨】ビットコインの価格上昇、８０万円台回復\u3000約１カ月半ぶり\u3000\u3000\u3000\u3000\u3000\u3000$BTC https://t.co/1OaM6ANOLX https://t.co/Ezd82kCt9L',
       'Stellar $XLM price: $0.297852 Binance registration is now OPEN for limited time! 💸 💰  ➡️… https://t.co/TteerEnNjo'],
      dtype=object)

In [131]:
sentiment_score=[]
cnt=0
for text in df['text'].values:
  try:
  
    sentiment_score.append(predict(text,model,vocab).data.numpy().flatten().tolist())
  except RuntimeError:
    cnt+=1
    sentiment_score.append(np.nan)
    pass


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:113: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [132]:
#tf=df[:100]

In [133]:
#np.array(sentiment_score).shape

In [134]:
df['sentiment_score']=sentiment_score

In [135]:
#sentiment_score=np.array(sentiment_score)

In [136]:
#sentiment_score.shape

In [137]:
sent=[]
for i in sentiment_score:
  try:
    sent.append(i.index(max(i)))
  except AttributeError:
    sent.append(np.nan)


In [138]:
df['sent']=sent

In [145]:
df[45:]

,id,text,timestamp,source,symbols,company_names,url,verified,sentiment_score,sent
45,1016108212276351000,$2.29 Billion in Sales Expected for Mosaic Co ...,Sun Jul 08 23:54:10 +0000 2018,WeekHerald,MOS,The Mosaic Company,http://weekherald.com/?p=2977936,False,"[0.041044894605875015, 0.12415481358766556, 0....",4.0
46,1016109247048945700,In 2 days I doubled my account with the option...,Sun Jul 08 23:58:17 +0000 2018,HarryPickens2,HSY,The Hershey Company,http://Optionsbypros.com,False,"[6.560474139405414e-05, 0.0009378290269523859,...",3.0
47,1016113566078947300,$NI high OI range is 26.00 to 26.00 for option...,Mon Jul 09 00:15:27 +0000 2018,optioncharts,NI,NiSource Inc.,http://maximum-pain.com/options/open-interest/NI,False,"[0.0015525782946497202, 0.009553798474371433, ...",3.0
48,1016114043638112300,RT @TradeSatoshi: $AMG AMGCHAIN will be delis...,Mon Jul 09 00:17:21 +0000 2018,BogdanoffJr,AMG,Affiliated Managers Group,NaN,False,"[0.0810590535402298, 0.24988409876823425, 0.07...",4.0
49,1016114104870658000,Payments halted under Obamacare program https:...,Mon Jul 09 00:17:35 +0000 2018,Gary49148971,HCA,HCA Healthcare,https://seekingalpha.com/news/3368467?source=ansh,False,"[0.1036430224776268, 0.426114946603775, 0.1306...",1.0
...,...,...,...,...,...,...,...,...,...,...
28259,1019730088617635800,$FB : 29234a9c-7f08-4d5a-985f-cb1a5554ecf9,Wed Jul 18 23:46:13 +0000 2018,test5f1798,FB,Facebook,NaN,False,NaN,NaN
28260,1019730115524288500,【仮想通貨】ビットコインの価格上昇、８０万円台回復 約１カ月半ぶり $BTC ht...,Wed Jul 18 23:46:19 +0000 2018,keizai_toushi17,BTC,Bitcoin,http://keizai-toushi-navi.com/?p=26838,False,NaN,NaN
28261,1019730115805184000,RT @invest_in_hd: 'Nuff said! $TEL #telcoin #...,Wed Jul 18 23:46:19 +0000 2018,iad81,BTC,Bitcoin,https://twitter.com/CRYPTOVERLOAD/status/10178...,False,"[0.10046154260635376, 0.13328050076961517, 0.1...",4.0
28262,1019730117252341800,【仮想通貨】ビットコインの価格上昇、８０万円台回復 約１カ月半ぶり $BTC ht...,Wed Jul 18 23:46:20 +0000 2018,O8viWMyrCV6cBOZ,BTC,Bitcoin,http://true.velvet.jp/monexx/archives/2357,False,NaN,NaN


In [146]:
df.to_csv('/stocker_sent.csv')

In [141]:
print(df[['text','sentiment_score','sent']])

                                                    text  ... sent
0      VIDEO: “I was in my office. I was minding my o...  ...  3.0
1      The price of lumber $LB_F is down 22% since hi...  ...  1.0
2      Who says the American Dream is dead? https://t...  ...  3.0
3      Barry Silbert is extremely optimistic on bitco...  ...  3.0
4      How satellites avoid attacks and space junk wh...  ...  0.0
...                                                  ...  ...  ...
28259         $FB : 29234a9c-7f08-4d5a-985f-cb1a5554ecf9  ...  NaN
28260  【仮想通貨】ビットコインの価格上昇、８０万円台回復　約１カ月半ぶり　　　　　　$BTC ht...  ...  NaN
28261  RT @invest_in_hd: 'Nuff said!  $TEL #telcoin #...  ...  4.0
28262  【仮想通貨】ビットコインの価格上昇、８０万円台回復　約１カ月半ぶり　　　　　　$BTC ht...  ...  NaN
28263  Stellar $XLM price: $0.297852 Binance registra...  ...  3.0

[28264 rows x 3 columns]


In [ ]:
tf['text',]

In [ ]:
np.array(sentiment_score).reshape(100,1)

In [ ]:
tf['sentiment_score']=sentiment_score

In [ ]:
tf

In [ ]:
model = TextClassifer(len(vocab)+1, 200,1, 128, 5, 0)hidden = model.init_hidden(20)

In [ ]:
logps, hidden = model.forward(text_batch, hidden)